# Sample Research Notebook
_Objective: Refactor codes from this notebook in OOP style_

## 1. Load Dataset

- Data often comes either from `.csv` file or database
- For quick prototyping, we load the dataset from `.csv` file

In [1]:
import pandas as pd
df = pd.read_csv('../data/loans.csv')
df.head()

,loan_id,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_term,credit_history,property_area,loan_approval_status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,0.0,360,1,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360,1,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360,1,Urban,Y


In [ ]:
### Alternative method: Query dataset from database

# from sqlalchemy import create_engine  # !pip install sqlalchemy

# host = '<localhost, or heicoders_AI300_db_url>'
# user = '<username>'
# password = '<password>'
# db_name = 'playground'
# db = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}')

# df = pd.read_sql('SELECT * FROM loans', db)

## 2. Data Preprocessing

- Data Cleaning
- Feature Engineering

### 2.1 Clean Target Column

In [2]:
df['loan_approval_status'].head()

0    Y
1    N
2    Y
3    Y
4    Y
Name: loan_approval_status, dtype: object

In [3]:
def binarize(value):
    if value in ['Yes', 'Y']:
        return 1
    elif value in ['No', 'N']:
        return 0

df['loan_approval_status'] = df['loan_approval_status'].apply(binarize)
df['loan_approval_status'].head()

0    1
1    0
2    1
3    1
4    1
Name: loan_approval_status, dtype: int64

### 2.2 Clean `dependent` Column

In [4]:
df['dependents'].unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [5]:
df['dependents'].fillna('-1', inplace=True)
df['dependents'] = df['dependents'].str.extract(r'(-?\d+)')  # Extract numeric value
df['dependents'] = df['dependents'].astype(int)

df['dependents'].unique()

array([ 0,  1,  2,  3, -1])

### 2.3 Replace `NaN`s with default string values

In [6]:
df[df.isnull().any(axis=1)].tail()

,loan_id,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_term,credit_history,property_area,loan_approval_status
579,LP002888,Male,No,0,Graduate,NaN,3182.0,2917.0,161.0,360,1,Urban,1
588,LP002925,NaN,No,0,Graduate,No,4750.0,0.0,94.0,360,1,Semiurban,1
592,LP002933,NaN,No,3,Graduate,Yes,9357.0,0.0,292.0,360,1,Semiurban,1
600,LP002949,Female,No,3,Graduate,NaN,416.0,41667.0,350.0,180,0,Urban,0
601,LP002950,Male,Yes,0,Not Graduate,NaN,2894.0,2792.0,155.0,360,1,Rural,1


In [7]:
for col in ['gender', 'married', 'self_employed']:
    df[col].fillna('Unknown', inplace=True)

assert len(df[df.isnull().any(axis=1)]) == 0

### 2.4 Feature Engineering

In [8]:
df['total_income'] = df['applicant_income'] + df['coapplicant_income']
df['loan_amt_income_ratio'] = df['loan_amount'] / df['total_income']

### 2.5 Drop unnecessary features and split dataset into X, y

In [9]:
df.columns

Index(['loan_id', 'gender', 'married', 'dependents', 'education',
       'self_employed', 'applicant_income', 'coapplicant_income',
       'loan_amount', 'loan_term', 'credit_history', 'property_area',
       'loan_approval_status', 'total_income', 'loan_amt_income_ratio'],
      dtype='object')

In [10]:
df.head()

,loan_id,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_term,credit_history,property_area,loan_approval_status,total_income,loan_amt_income_ratio
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,0.0,360,1,Urban,1,5849.0,0.000000
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360,1,Rural,0,6091.0,0.021015
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360,1,Urban,1,3000.0,0.022000
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360,1,Urban,1,4941.0,0.024287
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360,1,Urban,1,6000.0,0.023500


In [11]:
# Split dataset into model input features and labels (aka target variable)
X = df.drop(columns=['loan_approval_status', 'loan_id'])
y = df['loan_approval_status']

## 3. Model Evaluation

* Use GridSearchCV to find optimal parameters for CatBoost model
* Use `train_test_split` to get a rough idea of offline metrics e.g. AUC, F1 score

_Bonus: Feel free to add on more experimental models where appropriate._

In [12]:
# Extract categorical columns programmatically

cat_features = list(X.select_dtypes(include='object').columns)
print(cat_features)

['gender', 'married', 'education', 'self_employed', 'property_area']


### 3.1 Hyperparameter Tuning

In [13]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

model = CatBoostClassifier()
param_grid = {  # simple example, feel free to tweak
    'iterations': [50, 100, 200],
    'depth': [4, 6, 8]
}
# Reference: https://catboost.ai/en/docs/concepts/parameter-tuning

grid_search = GridSearchCV(model, param_grid, cv=4)
grid_search.fit(X, y, cat_features=cat_features, verbose=False)

print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'depth': 8, 'iterations': 50}


### 3.2 Calculate offline metrics

* Area Under Curve (AUC)
* Precision, Recall, [F1](https://www.v7labs.com/blog/f1-score-guide) Score

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_model = CatBoostClassifier(**grid_search.best_params_, random_state=42)
best_model.fit(X_train, y_train, cat_features=cat_features, verbose=False)
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:,1]

# Evaluate the model with various metrics
print("AUC:", roc_auc_score(y_test, y_pred_proba))
print("Precision Score:", precision_score(y_test, y_pred))
print("Recall Score:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

AUC: 0.7967948717948719
Precision Score: 0.7622377622377622
Recall Score: 0.9083333333333333
F1 Score: 0.8288973384030418
